# Energy Consumption: Preprocessing & Simple Forecasting
This notebook preprocesses hourly energy consumption data, visualizes trends, implements three simple forecasting methods (Naive, Simple Average, Rolling Average), and evaluates them using MFE, MAE, and MAPE.

## 1. Import required libraries

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
print('Libraries imported')

Libraries imported


## 2. Load and inspect dataset

In [13]:
# Load the dataset
df = pd.read_csv('events.csv', header=0)

print('Dataset shape:', df.shape)
print('Columns:', df.columns.tolist())
display(df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'events.csv'

## 3. Preprocess: parse dates and set DatetimeIndex

In [ ]:
# Keep only relevant columns and rename to 'time' and 'energy'
df_clean = df[['Start time UTC+03:00', 'Electricity consumption in Finland']].copy()
df_clean.columns = ['time', 'energy']
# Parse and set index
df_clean['time'] = pd.to_datetime(df_clean['time'])
df_clean['energy'] = pd.to_numeric(df_clean['energy'], errors='coerce')
df_clean = df_clean.set_index('time').sort_index()
print('Preprocessed shape:', df_clean.shape)

## 4. Handle empty/missing inputs

In [ ]:
print('Missing before:', df_clean.isna().sum().to_dict())
# Interpolate time-based gaps then ffill/bfill as fallback
df_clean['energy'] = df_clean['energy'].interpolate(method='time', limit_direction='both')
df_clean['energy'] = df_clean['energy'].fillna(method='ffill').fillna(method='bfill')
print('Missing after:', df_clean.isna().sum().to_dict())

## 5. Detect and handle outliers

In [ ]:
Q1 = df_clean['energy'].quantile(0.25)
Q3 = df_clean['energy'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
out_mask = (df_clean['energy'] < lower) | (df_clean['energy'] > upper)
print('Outliers detected:', out_mask.sum())
# Replace outliers with 24-hour rolling median
if out_mask.any():
    rm = df_clean['energy'].rolling(window=24, center=True, min_periods=1).median()
    df_clean.loc[out_mask, 'energy'] = rm[out_mask]
    print('Replaced outliers with rolling median')
print(df_clean['energy'].describe())

## 6. Feature engineering: year, month, day

In [ ]:
df_clean['year'] = df_clean.index.year
df_clean['month'] = df_clean.index.month
df_clean['day'] = df_clean.index.day
df_clean['hour'] = df_clean.index.hour
df_clean['weekday'] = df_clean.index.weekday
display(df_clean.head())

## 7. Visualize trends by year, month and day

In [ ]:
import matplotlib.dates as mdates
fig, axes = plt.subplots(2,2, figsize=(14,10))
yearly = df_clean.groupby('year')['energy'].sum()
axes[0,0].bar(yearly.index, yearly.values); axes[0,0].set_title('Total by Year')
monthly = df_clean.groupby('month')['energy'].mean()
axes[0,1].plot(monthly.index, monthly.values, marker='o'); axes[0,1].set_title('Average by Month')
hourly = df_clean.groupby('hour')['energy'].mean()
axes[1,0].bar(hourly.index, hourly.values); axes[1,0].set_title('Average by Hour')
df_clean['weekday_name'] = df_clean.index.day_name()
weekday_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
data = [df_clean[df_clean['weekday_name']==d]['energy'].values for d in weekday_order]
axes[1,1].boxplot(data, labels=weekday_order); axes[1,1].set_title('Distribution by Weekday')
plt.tight_layout(); plt.show()

## 8. Time-based train/test split

In [ ]:
train_size = int(len(df_clean) * 0.8)
train = df_clean.iloc[:train_size].copy()
test = df_clean.iloc[train_size:].copy()
train_series = train['energy'].values
test_series = test['energy'].values
split_date = train.index[-1]
print('Train size:', len(train), 'Test size:', len(test))

## 9. Implement forecasting methods

In [ ]:
def naive_forecast(train_values, h):
    return np.full(h, train_values[-1])
def simple_average_forecast(train_values, h):
    return np.full(h, train_values.mean())
def rolling_average_forecast(train_values, h, window=24):
    return np.full(h, train_values[-window:].mean())
# Generate forecasts
h = len(test)
naive_pred = naive_forecast(train_series, h)
simple_pred = simple_average_forecast(train_series, h)
rolling_pred = rolling_average_forecast(train_series, h, window=24)
print('Forecasts computed')

## 10. Visualize forecasts on train and test sets

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,5))
ax.plot(train.index, train['energy'], label='Train', color='gray')
ax.plot(test.index, test['energy'], label='Test (actual)', color='blue')
ax.plot(test.index, naive_pred, label='Naive', linestyle='--', color='red')
ax.plot(test.index, simple_pred, label='Simple Avg', linestyle='--', color='green')
ax.plot(test.index, rolling_pred, label='Rolling Avg (24h)', linestyle='--', color='orange')
ax.axvline(split_date, color='k', linestyle=':')
ax.legend(); ax.set_title('Forecasts vs Actual')
plt.show()

## 11. Evaluation metrics: MFE, MAE, MAPE

In [ ]:
def mfe(y_true, y_pred):
    return np.mean(y_pred - y_true)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))
def mape(y_true, y_pred):
    eps = 1e-8
    mask = y_true != 0
    if mask.sum() == 0:
        return np.inf
    return 100 * np.mean(np.abs((y_pred[mask] - y_true[mask]) / (y_true[mask] + eps)))
# Compute metrics for each method
results = {}
results['Naive'] = {'MFE': mfe(test_series, naive_pred), 'MAE': mae(test_series, naive_pred), 'MAPE': mape(test_series, naive_pred)}
results['Simple Average'] = {'MFE': mfe(test_series, simple_pred), 'MAE': mae(test_series, simple_pred), 'MAPE': mape(test_series, simple_pred)}
results['Rolling Average (24h)'] = {'MFE': mfe(test_series, rolling_pred), 'MAE': mae(test_series, rolling_pred), 'MAPE': mape(test_series, rolling_pred)}
res_df = pd.DataFrame(results).T.round(4)
print(res_df)
res_df.to_csv('forecast_metrics.csv')
print('Saved metrics to forecast_metrics.csv')